In [1]:
import os
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"
import theano
import keras

RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

C:/ProgramData/Anaconda3/Library/mingw-w64/bin/../lib/gcc/x86_64-w64-mingw32/5.3.0/../../../../x86_64-w64-mingw32/bin/ld.exe: cannot open output file C:\Users\sreev\AppData\Local\Theano\compiledir_Windows-10-10.0.15063-SP0-Intel64_Family_6_Model_158_Stepping_9_GenuineIntel-3.5.4-64\cutils_ext\cutils_ext.pyd: Permission denied
collect2.exe: error: ld returned 1 exit status



00001	
00002	        #include "numpy/npy_3kcompat.h"
00003	        #include "numpy/arrayobject.h"
00004	        #include "theano_mod_helper.h"
00005	
00006	        extern "C"{
00007	        static PyObject *
00008	        run_cthunk(PyObject *self, PyObject *args)
00009	        {
00010	          PyObject *py_cthunk = NULL;
00011	          if(!PyArg_ParseTuple(args,"O",&py_cthunk))
00012	            return NULL;
00013	
00014	          if (!NpyCapsule_Check(py_cthunk)) {
00015	            PyErr_SetString(PyExc_ValueError,
00016	                           "Argument to run_cthunk must be a NpyCapsule.");
00017	            return NULL;
00018	          }
00019	          void * ptr_addr = NpyCapsule_AsVoidPtr(py_cthunk);
00020	          int (*fn)(void*) = (int (*)(void*))(ptr_addr);
00021	          void* it = NpyCapsule_GetDesc(py_cthunk);
00022	          int failure = fn(it);
00023	
00024	          return Py_BuildValue("i", failure);
00025	         }
00026	        #if NPY_API_VERSION >= 0x00

Exception: Compilation failed (return status=1): C:/ProgramData/Anaconda3/Library/mingw-w64/bin/../lib/gcc/x86_64-w64-mingw32/5.3.0/../../../../x86_64-w64-mingw32/bin/ld.exe: cannot open output file C:\Users\sreev\AppData\Local\Theano\compiledir_Windows-10-10.0.15063-SP0-Intel64_Family_6_Model_158_Stepping_9_GenuineIntel-3.5.4-64\cutils_ext\cutils_ext.pyd: Permission denied. collect2.exe: error: ld returned 1 exit status. 

In [ ]:
# Plot ad hoc CIFAR10 instances
from keras.datasets import cifar10
from matplotlib import pyplot
from scipy.misc import toimage
# load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [16]:
# Simple CNN model for CIFAR-10
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

In [17]:
# normalize inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [18]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [10]:
X_train = X_train.reshape(X_train.shape[0], 32 * 32 * 3)
X_test = X_test.reshape(X_test.shape[0], 32 * 32 * 3)

In [19]:
X_train.shape

(50000, 3, 32, 32)

In [21]:
# Create the model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32,), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
# Compile model
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_3 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 16, 16)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               4194816   
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
__________

In [22]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 332s - loss: 1.6749 - acc: 0.3952 - val_loss: 1.3929 - val_acc: 0.5063
Epoch 2/25
50000/50000 [==============================] - 334s - loss: 1.2971 - acc: 0.5340 - val_loss: 1.1607 - val_acc: 0.5930
Epoch 3/25
50000/50000 [==============================] - 330s - loss: 1.1353 - acc: 0.5927 - val_loss: 1.1013 - val_acc: 0.6062
Epoch 4/25
50000/50000 [==============================] - 330s - loss: 1.0199 - acc: 0.6354 - val_loss: 1.0149 - val_acc: 0.6372
Epoch 5/25
50000/50000 [==============================] - 340s - loss: 0.9316 - acc: 0.6689 - val_loss: 0.9951 - val_acc: 0.6477
Epoch 6/25
50000/50000 [==============================] - 346s - loss: 0.8562 - acc: 0.6953 - val_loss: 0.9565 - val_acc: 0.6575
Epoch 7/25
50000/50000 [==============================] - 348s - loss: 0.7840 - acc: 0.7219 - val_loss: 0.9301 - val_acc: 0.6720
Epoch 8/25
50000/50000 [=======================